In [1]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
user_name = os.getenv('user_name')
user_pwd = os.getenv('user_pwd')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
gcp_project = os.getenv('gcp_project')
host_name = os.getenv('host_name')
port = 5432
database = 'rain'


In [3]:
# llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo', openai_api_key=OPENAI_API_KEY)

### Pronto connection

In [3]:
pg_uri = f"postgresql+psycopg2://{user_name}:{user_pwd}@{host_name}:{port}/{database}"
db = SQLDatabase.from_uri(pg_uri, schema="bnk", include_tables=["transactions"])

In [7]:
question = "what is average transaction amount"

# Create db chain
QUERY = """
Given an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

{question}
"""

# Setup the database chain
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True, return_intermediate_steps=True)

question = QUERY.format(question=question)
res = db_chain(question)





> Entering new SQLDatabaseChain chain...

Given an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

what is average transaction amount

SQLQuery:SELECT AVG(amount) FROM bnk.transactions;
SQLResult: [(Decimal('118789.722724166730'),)]
Answer:The average transaction amount is 118789.722724166730.
> Finished chain.


In [39]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True, return_intermediate_steps=True)
res = db_chain("what is average transaction amount")



> Entering new SQLDatabaseChain chain...
what is average transaction amount
SQLQuery:SELECT AVG(amount) FROM bnk.transactions;
SQLResult: [(Decimal('118883.470281673050'),)]
Answer:The average transaction amount is 118883.470281673050.
> Finished chain.


In [10]:
for_output['intermediate_steps']

[{'input': "\nGiven an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.\nUse the following format:\n\nQuestion: Question here\nSQLQuery: SQL Query to run\nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nwhat is average transaction amount\n\nSQLQuery:SELECT AVG(amount) FROM bnk.transactions;\nSQLResult: [(Decimal('118789.722724166730'),)]\nAnswer:",
  'top_k': '5',
  'dialect': 'postgresql',
  'table_info': "\nCREATE TABLE bnk.transactions (\n\tid UUID NOT NULL, \n\tentity_id UUID NOT NULL, \n\ttype VARCHAR(6) NOT NULL, \n\tmethod VARCHAR(50) NOT NULL, \n\tamount BIGINT NOT NULL, \n\tchannel_fee BIGINT DEFAULT 0, \n\tinterchange_fee BIGINT DEFAULT 0, \n\tnetwork_fee BIGINT DEFAULT 0, \n\tnetwork VARCHAR(50), \n\tsource TEXT NOT NULL, \n\tdestination VARCHAR(256) NOT NULL, \n\texternal_id VARCHAR(256), \n\treference_id VARCHAR(256), \n\terror_code VARCHAR(50), \n\terror_message TEXT,

### Omnifin Connection

In [4]:
jdbcHostname = os.getenv('jdbcHostname')
jdbcUsername = os.getenv('jdbcUsername')
jdbcPassword = os.getenv('jdbcPassword')
dbms = os.getenv('dbms')

In [5]:
# import mysql.connector
# myconn = mysql.connector.connect(host = jdbcHostname, user = jdbcUsername, passwd = jdbcPassword, database = dbms) 
# print(myconn) 

In [6]:
import pymysql
from sqlalchemy import create_engine
dbcon = pymysql.connect(host=jdbcHostname, user=jdbcUsername, password=jdbcPassword, database=dbms)

uri = f'mysql+pymysql://{jdbcUsername}:{jdbcPassword}@{jdbcHostname}/{dbms}'
# engine = create_engine(uri, echo=True)

In [7]:
db = SQLDatabase.from_uri(uri, schema="beefin_uat", include_tables=["cr_pmnt_dtl"])

In [8]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True, return_intermediate_steps=True)

/Users/mayankaamseek/anaconda3/envs/IncredGPT/lib/python3.11/site-packages/langchain/chains/sql_database/base.py:63: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [9]:

QUERY = """
Given an input question, first create a syntactically correct mysql query to run, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

{question}
"""
question = "what was total payment amount on 2017-03-27"
question = QUERY.format(question=question)

In [10]:
res = db_chain(question)



> Entering new SQLDatabaseChain chain...

Given an input question, first create a syntactically correct mysql query to run, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

what was total payment amount on 2017-03-27

SQLQuery:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..


KeyboardInterrupt: 

In [ ]:
What tables are required to find ""

In [11]:
from langchain.agents import create_pandas_dataframe_agent
agent = create_pandas_dataframe_agent(OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY),iam, verbose=True)
agent("how many records are there? Give me rows and columns")
agent("Are there any missing values in columns")
agent("Are there any missing values in columns")



> Entering new AgentExecutor chain...
Thought: I need to check if there are any missing values in the columns
Action: python_repl_ast
Action Input: df.isnull().sum()
Observation: id                            0
full_name                     0
first_name                    0
last_name                     0
status                        0
email                         8
phone_number                  0
created_at                    0
updated_at                    0
activated_at                  9
source_code                   0
source_id                   100
metadata                      0
username                      0
document_number               1
external_employee_id        100
phone_number_verified_at    100
dtype: int64
Thought: I now know the final answer
Final Answer: Yes, there are missing values in some of the columns.

> Finished chain.


{'input': 'Are there any missing values in columns',
 'output': 'Yes, there are missing values in some of the columns.'}